In [2]:
#### Libs ###
from typing_extensions import TypedDict
from typing import Annotated, Dict, Any
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, END
from langchain_core.messages import AIMessage
import requests
import time
import pyodbc
from contextlib import closing
import pandas as pd
from warnings import filterwarnings
import requests
filterwarnings('ignore')

In [3]:
class config:
    conn_str = (
        "DRIVER={SQL Server};"
        "SERVER=216.48.191.98;"
        "DATABASE=Agentic_Automation;"
        "UID=ibsadmin;"
        "PWD=Viking@@ibs2023;"
    )
conn=pyodbc.connect(config.conn_str)
cur=conn.cursor()

In [4]:
# iterate task processing by order task_insert_time, task_number where task_Status ='New'
# check previous task of same webtop is completed if yes than run or pick next record

In [7]:

query_pick_records='''
select tm.CloudFlowApi,tp.Webtop_ID,tm.Process_Name,tp.Work_ID,tm.Task_Name,tp.Task_Number from Credit_GPT.dbo.Task_Processing as tp 
--left join Credit_GPT.dbo.[Task_Dependencies] as td on tp.Task_Number=td.Task_Number 
left join Credit_GPT.dbo.Task_Master as tm on tp.Task_Number=tm.Task_Number
where tp.Task_Status='New' --or tp.Task_Status'In Progress'
order by tp.task_insert_time,tp.Task_Number 
'''
def check_dependency_record_status(task_number,webtop_id):
    query_dependency_record=f'''
select Task_Number,Task_Name,Task_Status,Failure_Reason from Credit_GPT.dbo.Task_Processing where Task_Number in 
(select Dependency_Task_Number from Credit_GPT.dbo.Task_Dependencies where Webtop_ID='{webtop_id}' and Task_Number='{task_number}') 
'''
    df_dependency_records=pd.read_sql(query_dependency_record,conn)
    
    if len(df_dependency_records)==0:
        dependency_record_Status=("No Dependency tasks is there",df_dependency_records)
        return dependency_record_Status

    df_dependency_completed_records=df_dependency_records[df_dependency_records['Task_Status']=="Completed"]

    if len(df_dependency_completed_records)==len(df_dependency_records):
        dependency_record_Status=("Dependency tasks Are Completed",df_dependency_records)
    else:
        dependency_record_Status=("Dependency tasks Are Not Completed",df_dependency_records)
    return dependency_record_Status


In [8]:
# check_previous_record_status('3','ABC')
df_pick_records=pd.read_sql(query_pick_records,conn)
df_pick_records

,CloudFlowApi,Webtop_ID,Process_Name,Work_ID,Task_Name,Task_Number
0,https://prod-30.centralindia.logic.azure.com:4...,359PZ8600833,Credit_Ops,abaf042d-0009-4737-b7f3-4f0bbfaa117d,cas_preparation,CO_008
1,https://prod-07.centralindia.logic.azure.com:4...,359PZ8600833,Credit_Ops,abaf042d-0009-4737-b7f3-4f0bbfaa117d,crm_scraping,CO_002
2,https://prod-12.centralindia.logic.azure.com:4...,359PZ8600833,Credit_Ops,abaf042d-0009-4737-b7f3-4f0bbfaa117d,offer_check,CO_003
3,https://prod-29.centralindia.logic.azure.com:4...,359PZ8600833,Credit_Ops,abaf042d-0009-4737-b7f3-4f0bbfaa117d,data_extraction,CO_004
4,https://prod-02.centralindia.logic.azure.com:4...,359PZ8600833,Credit_Ops,abaf042d-0009-4737-b7f3-4f0bbfaa117d,external_checks,CO_005
5,https://prod-14.centralindia.logic.azure.com:4...,359PZ8600833,Credit_Ops,abaf042d-0009-4737-b7f3-4f0bbfaa117d,internal_checks,CO_006
6,https://prod-02.centralindia.logic.azure.com:4...,359PZ8600833,Credit_Ops,abaf042d-0009-4737-b7f3-4f0bbfaa117d,data_validation,CO_007


In [9]:
# url=df_pick_records.loc[0,'CloudFlowApi']
# print(url)
# body={
#         "Webtop_ID": "349PZ8633504",
#         "Process_Name": "Bureau Analysis",
#         "Work_ID": "work1",
#         "Task_Name":"DocDownload"
# }
# response=requests.post(url=url,json=body)
# response.raise_for_status()
# print(response)
def run_flow(url,webtop,process_name,work_id,task_name):
    body={
            "Webtop_ID":webtop,
            "Process_Name":process_name,
            "Work_ID":work_id,
            "Task_Name":task_name
            }
    response=requests.post(url,json=body)
    response.raise_for_status()
    return response

In [ ]:
while True:
    df_pick_records=pd.read_sql(query_pick_records,conn)
    for _,row in df_pick_records.iterrows():
        CloudFlowApi=row['CloudFlowApi']
        Webtop_ID=row['Webtop_ID']
        Process_Name=row['Process_Name']
        Work_ID=row['Work_ID']
        Task_Name=row['Task_Name']
        Task_Number=row['Task_Number']
        print(Task_Name,end=" ")
        previous_record_status,df=check_dependency_record_status(Task_Number,Webtop_ID)
        if previous_record_status=='Dependency tasks Are Completed' or previous_record_status=="No Dependency tasks is there":
            print(previous_record_status)
            print(f"==>{Task_Name} is Started for {Webtop_ID}, for Process {Process_Name} and Work_ID is {Work_ID}")
            response=run_flow(CloudFlowApi,Webtop_ID,Process_Name,Work_ID,Task_Name)
            cur.execute(f"update Credit_GPT.dbo.Task_Processing set Task_Status='In Progress' where Task_Number='{Task_Number}' and Webtop_ID='{Webtop_ID}'")
            conn.commit()
        else:
            print(previous_record_status)
            print(df)
            time.sleep(5)

            continue
        time.sleep(5)




        
    

cas_preparation Dependency tasks Are Not Completed
  Task_Number        Task_Name Task_Status Failure_Reason
0      CO_007  data_validation         New           None
crm_scraping No Dependency tasks is there
==>crm_scraping is Started for 359PZ8600833, for Process Credit_Ops and Work_ID is abaf042d-0009-4737-b7f3-4f0bbfaa117d
offer_check Dependency tasks Are Not Completed
  Task_Number     Task_Name  Task_Status Failure_Reason
0      CO_002  crm_scraping  In Progress           None
data_extraction Dependency tasks Are Completed
==>data_extraction is Started for 359PZ8600833, for Process Credit_Ops and Work_ID is abaf042d-0009-4737-b7f3-4f0bbfaa117d
external_checks Dependency tasks Are Not Completed
  Task_Number        Task_Name  Task_Status Failure_Reason
0      CO_002     crm_scraping  In Progress           None
1      CO_004  data_extraction  In Progress           None
data_validation Dependency tasks Are Not Completed
  Task_Number        Task_Name  Task_Status Failure_Reason
0   